# 論文 11：使用膨脹卷積的多尺度上下文聚合
## Fisher Yu, Vladlen Koltun (2015)

### 膨脹/空洞卷積用於大感受野

擴大感受野而不損失解析度或增加參數！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 標準卷積 vs 膨脹卷積

**標準**：連續的核  
**膨脹**：帶間隙的核（膨脹率）

In [ ]:
def dilated_conv1d(input_seq, kernel, dilation=1):
    """
    一維膨脹卷積
    
    dilation=1：標準卷積
    dilation=2：跳過每隔一個位置
    dilation=4：跳過 3 個位置
    """
    input_len = len(input_seq)
    kernel_len = len(kernel)
    
    # 帶膨脹的有效核大小
    effective_kernel_len = (kernel_len - 1) * dilation + 1
    output_len = input_len - effective_kernel_len + 1
    
    output = []
    for i in range(output_len):
        # 應用膨脹核
        result = 0
        for k in range(kernel_len):
            pos = i + k * dilation
            result += input_seq[pos] * kernel[k]
        output.append(result)
    
    return np.array(output)

# 測試
signal = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
kernel = np.array([1, 1, 1])

out_d1 = dilated_conv1d(signal, kernel, dilation=1)
out_d2 = dilated_conv1d(signal, kernel, dilation=2)
out_d4 = dilated_conv1d(signal, kernel, dilation=4)

print(f"輸入：{signal}")
print(f"核：{kernel}")
print(f"\n膨脹率=1（標準）：{out_d1}")
print(f"膨脹率=2：{out_d2}")
print(f"膨脹率=4：{out_d4}")
print(f"\n感受野隨膨脹率呈指數增長！")

## 視覺化感受野

In [ ]:
# 視覺化膨脹如何影響感受野
fig, axes = plt.subplots(3, 1, figsize=(14, 8))

for ax, dilation, title in zip(axes, [1, 2, 4], 
                                ['膨脹率=1（標準）', '膨脹率=2', '膨脹率=4']):
    # 顯示使用了哪些位置
    positions = [0, dilation, 2*dilation]
    
    ax.scatter(range(10), signal, s=200, c='lightblue', edgecolors='black', zorder=2)
    ax.scatter(positions, signal[positions], s=300, c='red', edgecolors='black', 
              marker='*', zorder=3, label='核使用的位置')
    
    # 繪製連接
    for pos in positions:
        ax.plot([pos, pos], [0, signal[pos]], 'r--', alpha=0.5, linewidth=2)
    
    ax.set_title(f'{title} - 感受野：{1 + 2*dilation} 個位置')
    ax.set_xlabel('位置')
    ax.set_ylabel('值')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-0.5, 9.5)

plt.tight_layout()
plt.show()

## 二維膨脹卷積

In [ ]:
def dilated_conv2d(input_img, kernel, dilation=1):
    """
    二維膨脹卷積
    """
    H, W = input_img.shape
    kH, kW = kernel.shape
    
    # 有效核大小
    eff_kH = (kH - 1) * dilation + 1
    eff_kW = (kW - 1) * dilation + 1
    
    out_H = H - eff_kH + 1
    out_W = W - eff_kW + 1
    
    output = np.zeros((out_H, out_W))
    
    for i in range(out_H):
        for j in range(out_W):
            result = 0
            for ki in range(kH):
                for kj in range(kW):
                    img_i = i + ki * dilation
                    img_j = j + kj * dilation
                    result += input_img[img_i, img_j] * kernel[ki, kj]
            output[i, j] = result
    
    return output

# 建立帶模式的測試圖像
img = np.zeros((16, 16))
img[7:9, :] = 1  # 水平線
img[:, 7:9] = 1  # 垂直線（十字）

# 3x3 邊緣檢測核
kernel = np.array([[-1, -1, -1],
                   [-1,  8, -1],
                   [-1, -1, -1]])

# 使用不同膨脹率應用
result_d1 = dilated_conv2d(img, kernel, dilation=1)
result_d2 = dilated_conv2d(img, kernel, dilation=2)

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('輸入圖像')
axes[0].axis('off')

axes[1].imshow(result_d1, cmap='RdBu')
axes[1].set_title('膨脹率=1（3x3 感受野）')
axes[1].axis('off')

axes[2].imshow(result_d2, cmap='RdBu')
axes[2].set_title('膨脹率=2（5x5 感受野）')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("更大的膨脹率 → 更大的感受野 → 捕捉更廣的上下文")

## 多尺度上下文模組

In [ ]:
class MultiScaleContext:
    """堆疊遞增膨脹率的膨脹卷積"""
    def __init__(self, kernel_size=3):
        self.kernel_size = kernel_size
        
        # 為每個尺度建立核
        self.kernels = [
            np.random.randn(kernel_size, kernel_size) * 0.1
            for _ in range(4)
        ]
        
        # 膨脹率：1, 2, 4, 8
        self.dilations = [1, 2, 4, 8]
    
    def forward(self, input_img):
        """
        應用多尺度膨脹卷積
        """
        outputs = []
        
        current = input_img
        for kernel, dilation in zip(self.kernels, self.dilations):
            # 應用膨脹卷積
            out = dilated_conv2d(current, kernel, dilation)
            outputs.append(out)
            
            # 填充回原始大小（簡化版）
            pad_h = (input_img.shape[0] - out.shape[0]) // 2
            pad_w = (input_img.shape[1] - out.shape[1]) // 2
            current = np.pad(out, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
            
            # 裁剪以匹配輸入大小
            current = current[:input_img.shape[0], :input_img.shape[1]]
        
        return outputs, current

# 測試多尺度
msc = MultiScaleContext(kernel_size=3)
scales, final = msc.forward(img)

print(f"每層的感受野：")
for i, d in enumerate(msc.dilations):
    rf = 1 + 2 * d * (len(msc.dilations) - 1)
    print(f"  層 {i+1}（膨脹率={d}）：{rf}x{rf}")

## 關鍵要點

### 膨脹卷積：
- 在核權重之間插入零（空洞）
- **感受野**：$(k-1) \cdot d + 1$，其中 $k$=核大小，$d$=膨脹率
- **相同參數**：與標準卷積一樣
- **更大上下文**：無需池化

### 優勢：
- ✅ 感受野指數增長
- ✅ 無解析度損失（相比池化）
- ✅ 相同參數數量
- ✅ 多尺度上下文聚合

### 應用：
- **語義分割**：密集預測任務
- **音訊生成**：WaveNet
- **時間序列**：TCN（時間卷積網路）
- **任何需要大感受野的任務**

### 比較：
| 方法 | 感受野 | 解析度 | 參數 |
|--------|----------------|------------|------------|
| 標準卷積 | 小 | 完整 | 低 |
| 池化 | 大 | 降低 | 低 |
| 大核 | 大 | 完整 | 高 |
| **膨脹卷積** | **大** | **完整** | **低** |